In [ ]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.core.frame import DataFrame
import pandas as pd
import numpy as np
import xlrd
%matplotlib inline
warnings.filterwarnings("ignore")
sns.set()
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 空气质量分指数级别与浓度对应表
df_level = pd.read_excel('空气质量分指数级别.xlsx')

In [ ]:
# IAQI计算
def IAQI(C_p, column):
    df_IAQI = df_level['IAQI']
    df_max = max(df_level[column])

    if column != 'O3':
        for i in range(len(df_level[column])):
            if i > 0:
                j = i - 1
            else:
                j = i
            if C_p == df_level.loc[i, column]:
                IAQI = df_IAQI[i]
            elif C_p > df_level.loc[j, column] and C_p < df_level.loc[i, column]:
                IAQI_Hi = df_IAQI[i]
                IAQI_Lo = df_IAQI[j]
                BP_Hi = df_level.loc[i, column]
                BP_Lo = df_level.loc[j, column]
                IAQI = ((IAQI_Hi - IAQI_Lo)*(C_p - BP_Lo)) / (BP_Hi - BP_Lo) + IAQI_Lo
            else:
                if C_p == df_max:
                    IAQI = 500
                else:
                    continue
    else:
        for i in range(0, 6):
            if i > 0:
                j = i - 1
            else:
                j = i
            if C_p == df_level.loc[i, column]:
                IAQI = df_IAQI[i]
            elif C_p > df_level.loc[j, column] and C_p < df_level.loc[i, column]:
                IAQI_Hi = df_IAQI[i]
                IAQI_Lo = df_IAQI[j]
                BP_Hi = df_level.loc[i, column]
                BP_Lo = df_level.loc[j, column]
                IAQI = ((IAQI_Hi - IAQI_Lo)*(C_p - BP_Lo)) / (BP_Hi - BP_Lo) + IAQI_Lo
            else:
                if C_p == df_max:
                    IAQI = 300
                else:
                    continue
    return IAQI

In [ ]:
# O3最大8小时平均浓度计算
def C_O3(df_question1_hour_O3,n):
    C_O3 = []
    for b in [df_question1_hour_O3[i:i + n] for i in range(0, len(df_question1_hour_O3), n)]:
        df_question1_hour_O3_everyday = b
        c_t = []
        for t in range(8,25):
            i = t - 8
            c_i = []
            for i in range(i,t):
                c_i_8h = df_question1_hour_O3_everyday[i]
                c_i.append(c_i_8h)
            c_t_8h = 1/8*sum(c_i)
            c_t.append(c_t_8h)

        C_O3_day = max(c_t)
        C_O3.append(C_O3_day)
        
    return C_O3

In [ ]:
# 直接根据日实测值计算日AQI
def AQI_day(data):
    columns=['SO2','NO2','PM10','PM2.5','O3','CO']
    AQI = []
    for i in range(len(data)):
        IAQI_p = []
        for column in columns:
            C_p = data[column][i]
            df_icolumn = IAQI(C_p,column)
            IAQI_p.append(df_icolumn)
        IAQI_p_max = max(IAQI_p)
        AQI.append(IAQI_p_max)

    return AQI

In [ ]:
# 根据小时实测值计算日实测值
def data_hour_day(data,n):
    # 数据准备
    data['监测时间'] = pd.to_datetime(data['监测时间'])
    data = data.set_index(['监测时间'], drop=True)
    df_hour_O3 = data['O3'].copy()
    data.drop(columns=['地点','O3'],inplace=True)
    
    # 计算日均值
    df_hour_to_day = data.resample('D').mean()
    
    # 计算O3日值
    df_hour_O3 = df_hour_O3
    O3_C = C_O3(df_hour_O3,n)
    df_hour_to_day.insert(5,'O3',O3_C)
    df_hour_to_day['地点'] = '监测点A'
    
    return df_hour_to_day

# Question 1

In [ ]:
# 日实测值计算AQI
df_question1_day = pd.read_excel('问题1.xlsx',sheet_name='日实测值')
df_question1_day.rename(columns={'SO2监测浓度(μg/m³)': 'SO2', 'NO2监测浓度(μg/m³)': 'NO2', 'PM10监测浓度(μg/m³)': 'PM10',
                                 'PM2.5监测浓度(μg/m³)': 'PM2.5', 'O3最大八小时滑动平均监测浓度(μg/m³)': 'O3', 'CO监测浓度(mg/m³)': 'CO'}, inplace=True)
dayDataAQI = AQI_day(df_question1_day)
print(dayDataAQI)
>>>[60.0, 46.0, 108.18181818181819, 137.27272727272728]

In [ ]:
# 小时实测值计算AQI
df_question1_hour = pd.read_excel('问题1.xlsx',sheet_name='小时实测值')
df_question1_hour.rename(columns={'SO2监测浓度(μg/m³)': 'SO2', 'NO2监测浓度(μg/m³)': 'NO2', 'PM10监测浓度(μg/m³)': 'PM10',
                                 'PM2.5监测浓度(μg/m³)': 'PM2.5', 'O3监测浓度(μg/m³)': 'O3', 'CO监测浓度(mg/m³)': 'CO'}, inplace=True)
data_day_hour = df_question1_hour
n = 24
df_hour_to_day = data_hour_day(data_day_hour,n)
df_hour_to_day.drop(columns=['地点'],axis=1,inplace=True)
df_hour_to_day= df_hour_to_day.astype('int')
hourDataAQI = AQI_day(df_hour_to_day)
print(hourDataAQI)
>>>[60.0, 46.0, 108.18181818181819, 136.36363636363637]

# Question 2

数据补全

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_excel('附件1 监测点A空气质量预报基础数据.xlsx',sheet_name='监测点A逐小时污染物浓度与气象实测数据')

In [ ]:
df_time = df['监测时间'].copy()

df['监测时间'] = pd.to_datetime(df['监测时间'])
df = df.set_index(['监测时间'], drop=True)

df.drop(columns=['地点'],inplace=True)

columns = df.columns
for column in columns:
    miss_rate = np.sum(df[column].isnull())/len(df[column])
    print("missing rate of column %s is %f" %(column,miss_rate))

In [ ]:
columns = df.columns
m = '—'
for column in columns:
    df[column][df[column] == m] = None
    df[column] = df[column].astype('float')
    df[column][df[column] == 0] = None
    df[column][df[column] < 0] = None
    miss_rate = np.sum(df[column].isnull())/len(df[column])
    print("missing rate of column %s is %f" %(column,miss_rate))

In [ ]:
print(df.isnull().sum())

In [ ]:
df_copy = df.copy()
df_copy.drop(columns = ['O3监测浓度(μg/m³)'],inplace = True) 

In [ ]:
## 定义KNN
def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out

In [ ]:
# 污染物浓度补全（除O3）
columns = df_copy.columns
for column in columns:
    df_copy[column] = knn_mean(df_copy[column], 12)
    miss_rate = np.sum(df_copy[column].isnull())/len(df_copy[column])
    print("missing rate of column %s is %f" %(column,miss_rate))

In [ ]:
# O3补全
df_copy_fill = df_copy.copy()
df_copy_fill.insert(0,'O3监测浓度(μg/m³)',df_copy_O3) 
# 取出不为空的全部样本进行训练
known = df_copy_fill[df_copy_fill['O3监测浓度(μg/m³)'].notnull()]
print(known.shape) 
# 取出为空的样本用于填充空值
unknown = df_copy_fill[df_copy_fill['O3监测浓度(μg/m³)'].isnull()]
print(unknown.shape) #2078, 54

# 取填充列
y = known.iloc[:, 0]
y = np.array(y)
# 取除了填充列的其他列
X = known.iloc[:, 1:]
X = np.array(X)

rfr = RandomForestRegressor(random_state=666, n_estimators=2000, n_jobs=-1)
rfr.fit(X, y)

df_copy_fill.loc[(df_copy_fill['O3监测浓度(μg/m³)'].isnull()), 'O3监测浓度(μg/m³)'] = rfr.predict(unknown.iloc[:, 1:])
df_copy_fill

In [ ]:
columns = df_copy_fill.columns
for column in columns:
    miss_rate = np.sum(df_copy_fill[column].isnull())/len(df_copy_fill[column])
    print("missing rate of column %s is %f" %(column,miss_rate))

In [ ]:
df_copy_fill.to_excel('附件1_监测点A小时实测值.xlsx')

### 滞后性分析

In [ ]:
df = pd.read_excel('附件1_监测点A小时实测值.xlsx')

In [ ]:
# 标准化数据
from sklearn import preprocessing

values = df.values #dataframe转换为array

standard_data = preprocessing.scale(values) 
standard_data = pd.DataFrame(standard_data) #将array还原为dataframe

standard_data.columns=df.columns #命名标题行

In [ ]:
# 差分
diff_truedata = df.diff()

diff_truedata.drop(columns=['风向(°)'],axis=1,inplace=True)
diff_truedata.drop([0],axis = 0,inplace = True)

In [ ]:
# 多元回归判断滞后阶数
import statsmodels.api as sm

In [ ]:
from sklearn import preprocessing

values = diff_truedata.values #dataframe转换为array

diff_standard_data = preprocessing.scale(values) 
diff_standard_data = pd.DataFrame(diff_standard_data) #将array还原为dataframe

diff_standard_data.columns=diff_truedata.columns #命名标题行

In [ ]:
X_diff = pd.concat([diff_standard_data['温度(℃)'],diff_standard_data['湿度(%)'],diff_standard_data['气压(MBar)'],diff_standard_data['风速(m/s)']],axis = 1)

x_diff = X_diff 
y_diff = diff_standard_data['O3监测浓度(μg/m³)'] 
model = sm.OLS(y_diff, x_diff) 
result = model.fit() 
result.summary()

In [ ]:
# 测试不同滞后阶数下气象条件对各污染物的影响
columns = ['SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)', 'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)']

mse_matrix = []
mse_matrix = pd.DataFrame(lag_mse_matrix)

for column in columns:
    lag_mse_matrix = []

    for lag_step in range(1, 101):

        diff_truedata_copy = diff_standard_data.copy()
        lag_AQI = diff_standard_data[column].values

        lag_huaf = []
        for i in range(lag_step):
            a = np.nan
            lag_huaf.append(a)

        lag_huaf = DataFrame(lag_huaf)
        lag_AQI = DataFrame(lag_AQI)
        lag_lag_step = lag_AQI.iloc[:-lag_step, :]
        lag_lag_step = DataFrame(lag_lag_step)
        lag_lag_step_final = pd.concat([lag_huaf, lag_lag_step], axis=0)
        lag_lag_step_final = lag_lag_step_final.reset_index(drop=True)

        diff_truedata_copy.insert(0, '滞后小时数', lag_lag_step_final)

        diff_truedata_copy = diff_truedata_copy[~(
            diff_truedata_copy['滞后小时数'].isnull())]

        X_diff = pd.concat([diff_truedata_copy['温度(℃)'], diff_truedata_copy['湿度(%)'],
                            diff_truedata_copy['气压(MBar)'], diff_truedata_copy['风速(m/s)']], axis=1)

        x_diff = X_diff
        y_diff = diff_truedata_copy['滞后小时数']
        model = sm.OLS(y_diff, x_diff)
        result = model.fit()

        current_mse = result.mse_model

        lag_mse_matrix.append(current_mse)
    
    mas_matrix[cloumn] = lag_mse_matrix

In [ ]:
mas_matrix = pd.DataFrame(mas_matrix)

df_lag_cor_matrix_O3 = mas_matrix.iloc[0:100]
df_lag_cor_matrix_SO2 = mas_matrix.iloc[100:200]
df_lag_cor_matrix_NO2 = mas_matrix.iloc[200:300]
df_lag_cor_matrix_PM10 = mas_matrix.iloc[300:400]
df_lag_cor_matrix_PM25 = mas_matrix.iloc[400:500]
df_lag_cor_matrix_CO = mas_matrix.iloc[500:600]

In [ ]:
plt.figure(figsize=(12,9),dpi=80) 
plt.plot(df_lag_cor_matrix_O3,label = 'O3',color='r')
plt.plot(df_lag_cor_matrix_SO2,label = 'SO2', color='orange')
plt.plot(df_lag_cor_matrix_NO2,label = 'NO2', color='purple')
plt.plot(df_lag_cor_matrix_PM10,label = 'PM10', color='g')
plt.plot(df_lag_cor_matrix_PM25,label = 'PM2.5', color='y')
plt.plot(df_lag_cor_matrix_CO,label = 'CO', color='b')
plt.legend(loc='best') #upper/lower left/right
plt.title("气象条件对污染物浓度的滞后影响误差") 
plt.xlabel('滞后步数')
plt.ylabel('模型MSE')
plt.show() 

根据构造特征和实测数据特征进行聚类

In [ ]:
df = pd.read_excel('新特征构造_监测点A小时实测值.xlsx')

In [ ]:
df.drop(columns=['监测时间'],axis=1,inplace=True)
train = np.array(df)
from sklearn.cluster import KMeans
from sklearn import preprocessing

kmeans = KMeans(n_clusters=3, random_state=None, algorithm='auto')
#min_max_scaler = preprocessing.MinMaxScaler()
#train_x = min_max_scaler.fit_transform(train)
kmeans.fit(train)

# 输出及聚类后的每个样本点的标签（即类别），预测新的样本点所属类别
print(kmeans.labels_)

In [ ]:
cluster = kmeans.labels_
train = DataFrame(train)
train.columns  = df.columns
train.insert(0,'cluster',cluster)

In [ ]:
train['cluster'].value_counts()

In [ ]:
kmeans_cluster_centers = kmeans.cluster_centers_
kmeans_cluster_centers = DataFrame(kmeans_cluster_centers)
kmeans_cluster_centers.columns = df.columns

In [ ]:
kmeans_cluster_centers.to_excel('聚类中心_kmeans_3类.xlsx')
train.to_excel('聚类结果_kmeans_3类.xlsx')

In [ ]:
cluster_result = pd.read_excel('聚类结果_kmeans_3类_可视化.xlsx')
columns = cluster_result.columns
for column in columns:
    miss_rate = np.sum(cluster_result[column].isnull())/len(cluster_result[column])
    print("missing rate of column %s is %f" %(column,miss_rate))

In [ ]:
cluster_result=cluster_result[~(cluster_result['old湿度(%)'].isnull())]

In [ ]:
 #进行数据降维  从而可视化
from sklearn.manifold import TSNE

tsne = TSNE()
m = tsne.fit_transform(cluster_result)
m = pd.DataFrame(m)
m.columns = ['特征1','特征2']

In [ ]:
df_cluster_result = cluster_result['cluster'].copy()
df_cluster_result = pd.DataFrame(df_cluster_result)

In [ ]:
m.insert(0,'cluster',df_cluster_result) 

In [ ]:
# 聚类结果可视化
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

plt.figure(figsize=(16,12),dpi=120)
#不同类别用不同颜色和样式绘图
d = m[m['cluster'] == 0]
plt.scatter(d['特征1'], d['特征2'], color = 'r',label = '聚类1',marker='.')
d = m[m['cluster'] == 1]
plt.scatter(d['特征1'], d['特征2'], color = 'g',label = '聚类2',marker='o')
d = m[m['cluster'] == 2]
plt.scatter(d['特征1'], d['特征2'], color = 'blue',label = '聚类3',marker='*')

plt.title('聚类结果降维')
plt.xlabel('降维特征1')
plt.ylabel('降维特征2')
plt.legend()

plt.show()

In [ ]:
# 聚类结果 特征可视化
columns = ['SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
           'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)']
for column in columns:
    var = 'cluster'
    boxdata = pd.concat([cluster_result[column], cluster_result[var]], axis=1)
    f, ax = plt.subplots(figsize=(8, 6))
    fig = sns.boxplot(x=var, y=column, data=boxdata)
    fig.axis()

# Question 3

In [ ]:
# 数据清洗与填充，代码同附件1中监测点A实测小时预处理方法
# 构造65小时前时间间隔的新时间特征列“近3天实测时间”
# 下面以监测点C的数据为例

In [ ]:
df1 = pd.read_excel('监测点C_一次预报.xlsx')
df2 = pd.read_excel('附件2_监测点C小时实测值.xlsx')

In [ ]:
df2.rename(columns={'监测时间': '近3天实测时间'}, inplace=True)

In [ ]:
merge1= pd.merge(df1,df2,on='近3天实测时间',how = 'left')

In [ ]:
df3 = df2.copy()
df3.drop(columns=['温度(℃)','湿度(%)','大气压(mBar)','风速(m/s)','风向(°)'],axis=1,inplace = True)
df3.rename(columns={'近3天实测时间': '预测时间', 'O3监测浓度(μg/m³)': 'true O3', 'SO2监测浓度(μg/m³)': 'true SO2',
                    'NO2监测浓度(μg/m³)': 'true NO2', 'PM10监测浓度(μg/m³)': 'true PM10', 
                    'PM2.5监测浓度(μg/m³)': 'true PM2.5', 'CO监测浓度(mg/m³)': 'true CO'}, inplace=True)

In [ ]:
merge2= pd.merge(merge1,df3,on='预测时间',how = 'left')

In [ ]:
merge2.to_excel('监测点C问题3数据.xlsx',index=False)

随机森林

In [ ]:
import sklearn.datasets as datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

In [ ]:
dataset = pd.read_excel('监测点A问题3数据.xlsx')
dataset=dataset[~(dataset['NO2监测浓度(μg/m³)'].isnull())]

In [ ]:
dataset_validition = dataset[-216:]
dataset_train = dataset[:-216]

In [ ]:
dataset_train=dataset_train[~(dataset_train['true O3'].isnull())]

In [ ]:
dataset_train.drop(columns=['地点','模型运行日期','预测时间','近3天实测时间'],axis=1,inplace = True)

X_data = dataset_train.copy()
X_data.drop(columns= ['true O3','true SO2','true NO2','true PM10','true PM2.5','true CO'],axis=1,inplace =True)
X_data

# 准备训练数据
# 因变量：true 污染物
X_zbl = X_data.values
y_ybl = dataset_train['true O3'].values

In [ ]:
X = X_zbl
y = y_ybl
print(len(X))
print(len(y))

In [ ]:
# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.05,
                                                    random_state=0)
regr = RandomForestRegressor(n_estimators=2000,criterion='mse',max_features='auto',max_depth=None,min_samples_split=2,min_samples_leaf=1)
# regr = RandomForestRegressor(random_state=100,
#                              bootstrap=True,
#                              max_depth=2,
#                              max_features=2,
#                              min_samples_leaf=3,
#                              min_samples_split=5,
#                              n_estimators=3)
pipe = Pipeline([('scaler', StandardScaler()), ('reduce_dim', PCA()),
                 ('regressor', regr)])
pipe.fit(X_train, y_train)

In [ ]:
dataset_validition.drop(columns=['地点','模型运行日期','预测时间','近3天实测时间'],axis=1,inplace = True)

validtion_data = dataset_validition.copy()
validtion_data.drop(columns= ['true O3','true SO2','true NO2','true PM10','true PM2.5','true CO'],axis=1,inplace =True)
validtion_data

# 准备验证数据
Xvalidtion_zbl = validtion_data.values
yvalidtion_ybl = dataset_validition['true O3'].values

In [ ]:
# O3的预测值为yvalidition_O3
yvalidition_O3 = pipe.predict(Xvalidtion_zbl)